# SPoOkY Meshes 👻

In [3]:
import base64
import time

import numpy as np
import scipy as sp
import meshplot as mp

import scipy.sparse
import scipy.sparse.linalg
import meshio
import IPython

In [4]:
def load_mesh(filename):
    m = meshio.read(filename)
    v = m.points
    f = m.cells[0].data
    return v, f


def save_mesh(filename, v, f):
    m  = meshio.Mesh(v, f)
    m.write(filename)


# v1, f1 = load_mesh('cw2_meshes/curvatures/plane.obj')
# v2, f2 = load_mesh('cw2_meshes/curvatures/lilium_s.obj')
# v3, f3 = load_mesh('cw2_meshes/decompose/armadillo.obj')
# v4, f4 = load_mesh('cw2_meshes/smoothing/fandisk_ns.obj')
# v5, f5 = load_mesh('cw2_meshes/smoothing/plane_ns.obj')
v7, f7 = load_mesh('sappho-hires.obj')

In [370]:
def standardize(v):
    """
    recenter points
    """
    v = v - np.mean(v, axis=0)
    v = v / np.std(v, axis=0).max()
    return v


def percentile_clip(h, n):
    """
    reject outlier points
    """
    return np.clip(h, np.percentile(h, n), np.percentile(h, 100 - n))

In [640]:
# Unsexy test mesh for debugging
v0 = np.array([
    [ 0, 0, 0],
    [-3, 4, 0],
    [-6, 0, 0],
    [-3,-4, 0],
    [ 3,-4, 0],
    [ 6, 0, 0],
    [ 3, 4, 0],
], dtype=np.float32)

f0 = np.array([
    [0, 1, 2],
    [0, 2, 3],
    [0, 3, 4],
    [0, 4, 5],
    [0, 5, 6],
    [0, 1, 6],
], dtype=np.int32)

# perturbed vertex indices to throw off naive algorithms
# f0 = np.array([
#     [1, 0, 2],
#     [2, 3, 0],
#     [0, 3, 4],
#     [4, 0, 5],
#     [5, 6, 0],
#     [6, 1, 0],
# ], dtype=np.int32)

## Laplace-Beltrami and Friends

### Vertex Area

In [1832]:
def normalize(v):
    """
    normalize rows of vectors
    """
    n = v.T / np.linalg.norm(v, axis=1)
    return n.T


def edges(f, n):
    """
    build an adjacency matrix, return edges in only one direction (upper triangular)
    """
    A = sp.sparse.triu(adjacency(f, len(v)))
    ei, ej = A.nonzero()
    ed = np.vstack([ei, ej]).T
    # sort for consistency
    return np.sort(ed, axis=0)


def adjacency(f, n):
    """
    build a sparse adjacency matrix from faces in coordinate format.
    faces have some redundancy which we use a bitmask to filter.
    """
    A = sp.sparse.dok_matrix((n, n), dtype=bool)
    i, j, k = f.T
    A[i, j] = True
    A[j, k] = True
    A[k, i] = True
    A = A.tocsr()
    A = A + A.T
    return A


def face_normals(v, f):
    """
    compute normals of faces f
    """
    i, j, k = f.T
    # compute edges
    e1 = v[j] - v[i]
    e2 = v[k] - v[i]
    # cross product is face normal
    n = np.cross(e1, e2, axis=1)
    return normalize(n)


def triangle_area(v, f):
    i, j, k = f.T
    a, b, c = v[i], v[j], v[k]

    ac = c - a
    bc = c - b

    return np.linalg.norm(np.cross(ac, bc, axis=1), axis=1) / 2


def vertex_area(v, f):
    """
    compute total area about vertices
    3.59ms for 281,724 faces, not bad son
    """
    n = len(v)
    A = np.zeros((3, len(f)))

    area = triangle_area(v, f)

    # set internal angles at vertex location in face array
    # using indexes that have duplicate values to increment doesn't work
    A[0] = area
    A[1] = area
    A[2] = area

    # some esoteric numpy for summing at duplicated indices
    # coo matrices are also an option
    data = A.ravel()
    cols = f.T.ravel()

    M = np.zeros(n)
    np.add.at(M, cols, data)

    return sp.sparse.diags(M)


def barycentric_mass(v, f):
    return vertex_area(v, f) / 3


# %timeit vertex_area(v4, f4)


M = barycentric_mass(v0, f0)
assert list(M.diagonal()) == [24, 8, 8, 8, 8, 8, 8]

assert len(edges(f0, len(v0))) == 12

In [1834]:
def edge_triangle_adjacency(f):
    """
    appendix B.3: Edge based gradient
    build an |E| x |F| adjacency matrix where each row sums to 0
    ~280ms for ~280000 faces, mostly due to sorting
    """
    i, j, k = f.T
    # edges
    ed = np.hstack([
        [i, j],
        [j, k],
        [k, i],
    ])
    # ensure a < b
    ed.sort(axis=0)
    # find indices of unique edges
    # rows are indices into unique edges
    ed, idx, rows = np.unique(ed, axis=1, return_index=True, return_inverse=True)
    # columns are triangle indices
    cols = np.tile(np.arange(len(f)), 3)
    # use indices to mask duplicate values, s.t. all rows sum to 1 for a closed manifold
    data = 0 - np.ones(len(rows), dtype=np.int8)
    data[idx] = 1
    # return edge-triangle adjacency matrix
    return scipy.sparse.coo_matrix((data, (rows, cols)))

# assert interior edges sum to 0, exterior edges sum to 1
assert np.allclose(edge_triangle_adjacency(f0).sum(axis=1).A.ravel(), [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1])

# Cotangent operator

In [871]:
def cotangent_curvature(v, f):
    """
    we can sum the contribution of all adjacent angles,
    then subtract the contribution of all interior angles.
    
    using edges would probably avoid some redundant work here (as above).

    133ms seconds for 281,724 faces. igl is 52.2ms.
    """
    n = len(v)
    
    # indices
    i, j, k = f.T
    a, b, c = v[i], v[j], v[k]

    # vectors
    ab = b - a
    ac = c - a
    bc = c - b
    
    # big chungus cotangent computation
    abc = np.einsum('ij,ij->i', ab, ac) / np.linalg.norm(np.cross( ab, ac, axis=1), axis=1)
    bac = np.einsum('ij,ij->i',-ab, bc) / np.linalg.norm(np.cross(-ab, bc, axis=1), axis=1)
    cab = np.einsum('ij,ij->i',-ac,-bc) / np.linalg.norm(np.cross(-ac,-bc, axis=1), axis=1)

    # set weights for opposite edges
    # a csr_matrix will sum the quantities for us!
    data = np.hstack([cab, abc, bac])
    rows = np.hstack([i,   j,   k, ])
    cols = np.hstack([j,   k,   i, ])
    T = sp.sparse.csr_matrix((data, (rows, cols)))

    # mad gains by flipping across the diagonal ;)
    T = T + T.T

    # Nearly there. sum the rows and use as diagonal.
    S = sp.sparse.diags(T.sum(axis=1).A.ravel())
    T = T - S

    # i'm never doing this again
    # divide by two as we have computed k_1 + k_2
    return T / 2


# %timeit cotangent_curvature(v7, f7)

C = cotangent_curvature(v0, f0)
T = np.diag(np.round((180 / np.pi) * C.todense()))

assert list(T) == [-205, -73, -60, -73, -73, -60, -73]
assert np.sum(C) < 1e-6

### Check that mean curvature works

In [872]:
def mean_curvature_metric(v, f):
    M = barycentric_mass(v, f)
    C = cotangent_curvature(v, f)
    Mi = sp.sparse.diags(1 / M.diagonal())
    Hn = -Mi @ C @ v
    H  = np.linalg.norm(Hn, axis=1)
    return H


c7 = percentile_clip(mean_curvature_metric(v7, f7), 5)

mp.plot(v7, f7, c7)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(28.620000…

### crazy stuff

In [1172]:
def rows_in(a, b):
    """
    return mask of a if row is in b
    we need to cast each row as a bytestring to perform efficient set operations
    absolute shenanigans
    """
    a_rows = np.ascontiguousarray(a)
    b_rows = np.ascontiguousarray(b)
    a_rows = a_rows.view((a_rows.dtype.descr * a_rows.shape[1]))
    b_rows = b_rows.view((b_rows.dtype.descr * b_rows.shape[1]))
    return np.isin(a_rows, b_rows).ravel()


def rowhash(a):
    a_rows = np.ascontiguousarray(a)
    return a_rows.view((a_rows.dtype.descr * a_rows.shape[1])).ravel()


def edge_gradient(v, f):
    # we need a sparse matrix of size |E| x |F|
    # this is the "jump discontinuity" between shared edges
    # in practice this is just +-1 for edges that are members of the triangle,
    # with the sign representing direction.
    # the paper states that all rows should sum to 1
    # this is presumably only true for closed manifolds
    
    # stack forward and reverse edges
    ed = edges(f, len(v))
    ei = np.arange(len(ed))
    ed = np.vstack([ed, np.fliplr(ed)])
    ei = np.hstack([ei, ei])

    # build edges from corners
    # these are incomplete and contain duplicates
    ij = f[:, [0, 1]]
    jk = f[:, [1, 2]]
    ki = f[:, [2, 0]]
    
    # in reverse order, present the edges in reverse
    ji = f[::-1, [1, 0]]
    kj = f[::-1, [2, 1]]
    ik = f[::-1, [0, 2]]
    
    # stack them all, alternating between forward and opposite edges
    # ensure indexing scheme matches alternate scheme
    fe = np.vstack([ij, ji, jk, kj, ki, ik])
    fi = np.arange(0, len(f))
    fi = np.hstack([fi, fi[::-1]])
    fi = np.tile(fi, 3)

    # find indices of fe
    a, b, c = np.intersect1d(rowhash(ed), rowhash(fe), return_indices=True)
    print(np.vstack([ei[b], fi[c]]).T)


In [1173]:
edge_gradient(v0, f0)

[[0 6]
 [0 5]
 [0 4]
 [0 3]
 [0 2]
 [0 1]
 [1 6]
 [1 2]
 [2 3]
 [3 4]
 [4 5]
 [5 6]]
[[ 5  0]
 [ 4  1]
 [ 3  2]
 [ 2  3]
 [ 1  4]
 [ 0  5]
 [ 5  5]
 [ 7  0]
 [ 6  5]
 [ 4  1]
 [ 7  0]
 [ 8  1]
 [ 3  2]
 [ 8  1]
 [ 9  2]
 [ 2  3]
 [ 9  2]
 [10  3]
 [ 1  4]
 [10  3]
 [11  4]
 [ 0  4]
 [ 6  5]
 [11  4]]


### Compute Normals

In [ ]:
def decompose_normals(v, f, u0, alpha, n, nonuniform, g):
    """
    decomposeNormals: apply the spectral TV decomposition to a normal vector field f, defined on the triangles of the
    domain M, applying the algorithm 3 in the paper, where:
    - alpha is the maximum time of diffusion
    - nComp is the number fo spectral components to be computed
    - reductionParam is the scale parameter for alpha
    - nonuniform is a binary flag indicating whether alpha is scaled non-uniformily
    """
    # build the edge gradient and divergence operators on surfaces for TV
    G = edge_gradient(M);
    A = edges_legths(M);
    % AA=spdiag(1./calc_tri_areas(M));
    D=-K'*spdiag(A);

## TV Divergence & Gradient Operators

hack this together

$$
\mathbf{G} f\left(\mathbf{t}_{i j k}\right)=\left(\begin{array}{cc}
\mathbf{v}_{j}^{\top}-\mathbf{v}_{i}^{\top} \\
\mathbf{v}_{k}^{\top}-\mathbf{v}_{i}^{\top}
\end{array}\right)^{\top}\left(\begin{array}{cc}
\left\|e_{i j}\right\|^{2} & \left\langle e_{i j}, e_{i k}\right\rangle \\
\left\langle e_{i j}, e_{i k}\right\rangle & \left\|e_{i k}\right\|^{2}
\end{array}\right)\left(\begin{array}{l}
f\left(\mathbf{v}_{j}\right)-f\left(\mathbf{v}_{i}\right) \\
f\left(\mathbf{v}_{k}\right)-f\left(\mathbf{v}_{i}\right)
\end{array}\right)
$$

In [158]:
def twonormest(A):
    """
    square root of largest singular value of A?!?
    https://en.wikipedia.org/wiki/Matrix_norm
    """
    [e] = scipy.sparse.linalg.svds(A, k=1, return_singular_vectors=False)
    return np.sqrt(e)


def tv_gradient_operator(v, f, axis):
    # indices
    i, j, k = f.T

    # edges?
    e_ij = v[j] - v[i]
    e_ik = v[k] - v[i]
    
    # hstack and reshape into F x 2 x 3
    # TODO: can we remove this swapaxes?
    A = np.hstack([
        e_ij,
        e_ik,
    ]).reshape(len(f), 2, 3).swapaxes(1, 2)
    
    e_ij2 = np.sum(e_ij ** 2, axis=1)
    e_ik2 = np.sum(e_ik ** 2, axis=1)
    
    e_min = np.min(np.hstack([np.sqrt(e_ij2), np.sqrt(e_ik2)]))

    e_ijik = np.einsum('ij,ij->i', e_ij, e_ik)

    B = np.hstack([
        [e_ij2, e_ijik, e_ijik, e_ik2],             
    ]).reshape(2, 2, len(f)).T

    # our 'function' is the X coordinate for now
    C = np.array([
        v[j][:, axis] - v[i][:, axis],
        v[k][:, axis] - v[i][:, axis],
    ]).reshape(len(f), 2, 1)

    D = A @ B @ C
    
    rows = np.repeat(np.arange(0, len(f)), 3)
    cols = f.ravel()
    data = D.ravel()

    G = sp.sparse.csr_matrix((data, (rows, cols)))
    
    # compute tau, sigma while we're in toon
    N = twonormest(G)
    t = e_min / N
    
    return G, t


def tv_gradient_operators(v, f, axis=0):
    """
    return G and adjoint D
    """
    G, t = tv_gradient_operator(v, f, axis)
    A = barycentric_mass(v, f)
    T = sp.sparse.diags(triangle_area(v, f))
    Ai = sp.sparse.diags(1 / A.diagonal())
    D = -Ai @ G.T @ T
    return G, D, t


# Test that our unsexy mesh produces the correct value for the first row
G, t = tv_gradient_operator(v0, f0)
assert np.allclose(G.todense().A[0], [2169, -732, 0, 0, 0, 0, 0])

# test that we get a value for tau / sigma
assert np.allclose(t, 0.083425544)

In [377]:
v, f = v7, f7

# todo: standardize v?
u0 = face_normals(v, f)

sub, phi = decompose_normals(v, f, u0, alpha, 40, nonuniform=False, g=1)


In [348]:
def prox_f(q):
    """
    Equation (16)
    Projects q onto the l2 ball
    """
    qT = q.T
    qT_norm = np.linalg.norm(qT, axis=0)
    indices = qT_norm > 1
    qT[:, indices] = qT[:, indices] / qT_norm[indices]
    return qT.T

A = np.array([
    [0  ,   1,   2],  # normalized
    [1/2, 1/2, 1/2],  # unchanged
    [1/3, 1/3, 1/3],  # unchanged
    [1/4, 1/4, 1/4],  # unchanged
])
proj = prox_f(A)

assert np.allclose(proj[0], [0. , 0.4472136 , 0.89442719])
assert np.allclose(proj[1], A[1])
assert np.allclose(proj[2], A[2])
assert np.allclose(proj[3], A[3])

In [349]:
def prox_g(u, u_0, tau, alpha):
    """
    Equation (17)
    ???
    """
    return (u + (tau / alpha) * u_0) / (1 + (tau / alpha))

In [364]:
def pdhg(v, f, u, u_0, q, alpha, axis=0):
    # algorithm 2
    G, D, tau = tv_gradient_operators(v, f, axis)
    sigma = tau
    theta = 0.5
    
    while True:
        u_next = prox_g(u - tau * D @ q, u_0, tau, alpha)
        u_bar  = u_next + theta * (u_next - u)
        q_next = prox_f(q + sigma * G @ u_bar)
        
        print(np.linalg.norm(u_next - u))
        if np.linalg.norm(u_next - u) < 1e-6:
            break

        u = u_next
        q = q_next
    return u

In [369]:
def solve(v, f, axis):
    """
    The algorithm simply evolves the input signal by N discrete steps along the TV flow;
    each iteration moves a step forward, with diffusion time equal to α.
    As changes happen quickly for small t and tend to become slower for larger t
    we iteratively increase the step size α of the evolution.
    
    Subsequently, the spectral representation φt is constructed incrementally using finite differences,
    such that the integral of Eq. (12) becomes a simple weighted sum over the φt.
    We give selection strategies for α and N in Appendix B.4.
    """
    n = 5  # number of time steps ~= number of features?!
    
    u_0 = v[:, axis]  # input signal
    u_t = np.zeros((n, *u_0.shape))  # feature stack
    p_t = np.zeros_like(u_t)  # spectral representation

    alpha = 1e-9  # step size - not sure what this should be
    
    # initialize u
    u = u_0
    
    # initialize q
    q = np.zeros_like(f[:, axis])

    for t in range(0, n):
        u_t[t] = pdhg(v, f, u, u_0, q, alpha, axis=0)
        p_t[t] = u_t[t] - u_t[t-1]  # 0 on first run?
        alpha  = alpha + alpha * 0.1
        u_0    = u_t[t]
    return p_t


p_t = solve(v7, f7, axis=0)

v7_t = np.vstack([p_t[0], v7[:, 1], v7[:, 2]]).T

print(v7_t)

mp.plot(v7_t, f7)

1.0948308170645484e-12
1.0822538358357315e-12
1.040196331096403e-12
9.876057739526858e-13
9.903753955288679e-13
[[ 23.082001  96.296005 -18.754002]
 [ 23.042002  96.040001 -18.974001]
 [ 23.162001  96.074005 -18.496   ]
 ...
 [ 24.982     78.830002 -16.602001]
 [ 55.364002  92.348007  -6.774   ]
 [ 55.476002  92.530006  -6.914001]]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(28.620000…